In [1]:
import pickle
import sys
import os

import numpy as np

# Get the path of the current script (or current working directory)
parent_dir = os.path.dirname(os.getcwd())
# Append the parent directory to sys.path
sys.path.append(parent_dir)

import src
from src.PrePipelineSteps import Pycromanager2NativeDataType, Make_Output_Dir_JF, Make_Analysis_Dir_JF, \
                                    ConsolidateImageShapes, TrimZSlices, AutomaticSpotDetection_JF

from src.PipelineSteps import CellSegmentationStepClass_JF, BIGFISH_SpotDetection

from src.PostPipelineSteps import BuildPDFReport, SaveSpotDetectionResults, SaveMasksToAnalysis, SendAnalysisToNAS

from src import Experiment, PipelineSettings, ScopeClass, PipelineDataClass, Pipeline

def save_dict_to_file(dic, location):
    f = open(location,'w')
    f.write(str(dic))
    f.close()

In [2]:
pipeline_package_name = 'Standard_Pipeline'

# Initialize Dataclasses

In [3]:
settings = PipelineSettings(return_data_to_NAS=1,
    optimization_segmentation_method='default',
    save_all_imgs=1,
    save_all_filtered_imgs=0,
    threshold_for_spot_detection=None,
    NUMBER_OF_CORES=4,
    save_filtered_images=1,
    remove_z_slices_borders=1,
    remove_out_of_focus_images=1,
    save_pdf_report=1,
    MINIMAL_NUMBER_OF_Z_SLICES_TO_CONSIDER_A_3D_IMAGE=5,
    NUMBER_Z_SLICES_TO_TRIM=0,
    user_select_number_of_images_to_run=100_000,  # TODO: This is bad but I want it to select all and am too lazy
                                                        # to deal with it rn
    minimum_spots_cluster=5,
    show_plots=1,
    CLUSTER_RADIUS=750,  # int(psf_yx*1.5)
    save_all_images=1,
    display_spots_on_multiple_z_planes=0,
    use_log_filter_for_spot_detection=0,
    sharpness_threshold=1.10,
    local_or_NAS=1,  # 0 for local, 1 for NAS
    save_files=1,
    model_nuc_segmentation=None,
    model_cyto_segmentation=None,
    pretrained_model_nuc_segmentation=None,
    pretrained_model_cyto_segmentation=None,
    diameter_nucleus=100,
    diameter_cytosol=200,
    save_masks_as_file=0,
    MINIMAL_NUMBER_OF_PIXELS_IN_MASK=1000,
    MAX_NUM_IMAGES_TO_AUTOMATICALLY_CALCULATE_THRESHOLD=20,
    MINUMUM_NUMBERs_IMAGES_TO_AUTOMATICALLY_CALCULATE_THRESHOLD=3,
    connection_config_location=r'/home/formanj/FISH_Processing_JF/FISH_Processing/config.yml', # "C:\Users\Jack\Desktop\config.yml"  '/home/formanj/FISH_Processing_JF/FISH_Processing/config.yml'
    share_name='share')

In [4]:
experiment = Experiment(initial_data_location=r"C:\Users\Jack\Desktop\test_data",
                        nucChannel=None,
                        cytoChannel=None,
                        FISHChannel=None,
                        voxel_size_z=300)

In [5]:
scope = ScopeClass( voxel_size_yx=130,
                    spot_z=500,
                    spot_yx=360,
                    microscope_saving_format='pycromanager'
                )

In [6]:
data = PipelineDataClass()

In [7]:
setting_dict = settings.__dict__
experiment_dict = experiment.__dict__
scope_dict = scope.__dict__


# Initialize Steps

In [8]:
PrePipelineSteps = [Pycromanager2NativeDataType(), 
                   Make_Output_Dir_JF(), 
                   Make_Analysis_Dir_JF(), 
                   ConsolidateImageShapes(), 
                   AutomaticSpotDetection_JF()]

In [9]:
PipelineSteps = [CellSegmentationStepClass_JF(), BIGFISH_SpotDetection()]

In [10]:
PostPipelineSteps = [BuildPDFReport(), SaveSpotDetectionResults(), SaveMasksToAnalysis(), SendAnalysisToNAS()]

# Pickle and Zip the Results

In [16]:
# Save the instances to pickle files
package_dir = os.path.join(os.getcwd(), 'Pipelines')
package_dir = os.path.join(package_dir, pipeline_package_name)
os.makedirs(package_dir, exist_ok=True)


settings_file = os.path.join(package_dir, 'settings.npy')
scope_file = os.path.join(package_dir, 'scope.npy')
experiment_file = os.path.join(package_dir, 'experiment.npy')
data_file = os.path.join(package_dir, 'data.pkl')
prepipeline_steps_file = os.path.join(package_dir, 'prepipeline_steps.pkl')
postpipeline_steps_file = os.path.join(package_dir, 'postpipeline_steps.pkl')
pipeline_steps_file = os.path.join(package_dir, 'pipeline_steps.pkl')

# np.save(settings_file, setting_dict)
# np.save(scope_file, scope_dict)
# np.save(experiment_file, experiment_dict)

save_dict_to_file(setting_dict, os.path.join(package_dir, 'settings.txt'))
save_dict_to_file(scope_dict, os.path.join(package_dir, 'scope.txt'))
save_dict_to_file(experiment_dict, os.path.join(package_dir, 'experiment.txt'))

# pickle.dump(settings, open(settings_file, 'wb'))
# pickle.dump(scope, open(scope_file, 'wb'))
# pickle.dump(experiment, open(experiment_file, 'wb'))
pickle.dump(data, open(data_file, 'wb'))
pickle.dump(PrePipelineSteps, open(prepipeline_steps_file, 'wb'))
pickle.dump(PostPipelineSteps, open(postpipeline_steps_file, 'wb'))
pickle.dump(PipelineSteps, open(pipeline_steps_file, 'wb'))



In [15]:
package_dir

'c:\\Users\\ericron.COLOSTATE\\GitHub\\FISH_Processing\\Pipelines\\Standard_Pipeline'